## <u>Сбор данных</u>

In [1]:
import requests
from bs4 import BeautifulSoup as BS

In [2]:
url = "https://www.avito.ru/moskva_i_mo/avtomobili?cd=1&p=1&q=bmw"
r = requests.get(url)
html = BS(r.content, "html.parser")

In [3]:
if html != 0:
    check = "Data received!"
    
print(check)

Data received!


In [4]:
r = requests.get(url)

print(r.status_code)  # Должен быть 200
print(r.headers)      # Информация о заголовках ответа
print(r.text)         # Выводит HTML содержимое страницы


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
page = 1

while True:
    
    url = ("https://www.avito.ru/moskva_i_mo/avtomobili?cd=1&p=" + str(page)+ "&q=bmw")
    r = requests.get(url)
    html = BS(r.content, "html.parser")
    pages = html.select(".items-items-kAJAg > .iva-item-root-_lk9K")
    
    if (len(pages)):
        for el in pages:
            # Получаем всю информацию об автомобиле
            info = el.find("div", {"class" : "iva-item-title-py3i_" }).find("a").get("title")

            # Выводим всю информацию об автомобиле
            # print(f'{info} \n')

            # Разделяем строку по запятым и пробелам
            parts = info.split(", ")

            # Получаем название автомобиля
            car_name = parts[0]

            # print("Название:", car_name)

            # Получаем год выпуска
            car_age = parts[1]

            #print("Год выпуска:", car_age)

            # Получаем километраж 
            car_km = parts[2]

            #print("Километраж", car_km)

            # Получаем стоимость
            car_price = parts[-1]


            price_start = car_price.find("цена") + 5
            price_end = car_price.find("руб")
            price = car_price[price_start:price_end].strip()

            #print("Стоимость:", price)

            #print(f'\n')
    else:
        break

## <u>Импортируем в csv-файл

In [9]:
import requests
from bs4 import BeautifulSoup as BS
import csv
import time

In [13]:
# Задаем настройки прокси
proxies = {
    'http': 'http://10.10.1.10:3128',
    'https': 'https://10.10.1.10:1080',
}

with open('cars_info.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Название', 'Год выпуска', 'Километраж', 'Стоимость'])

    page = 1
    while True:
        try:
            url = f"https://www.avito.ru/moskva_i_mo/avtomobili?cd=1&p={page}&q=bmw"
            r = requests.get(url, proxies=proxies)  # Добавляем прокси к запросу
            
            # Проверяем код состояния ответа
            if r.status_code == 429:
                retry_after = r.headers.get("Retry-After")
                delay = int(retry_after) if retry_after else 10
                print(f"Слишком много запросов, ждем {delay} секунд.")
                time.sleep(delay)
                continue  # Пропускаем оставшуюся часть цикла и повторяем запрос

            # Если код состояния не 429, то продолжаем обработку страницы
            r.raise_for_status()  # Вызовет исключение для кодов состояния 4xx/5xx
            html = BS(r.content, 'html.parser')
            pages = html.select('.items-items-kAJAg > .iva-item-root-_lk9K')
            
            if not pages:
                break  # Выходим из цикла, если нет контента

            for el in pages:
                info = el.find('div', {'class': 'iva-item-title-py3i_'}).find('a').get('title')
                parts = info.split(', ')
                car_name = parts[0]
                car_age = parts[1]
                car_km = parts[2]
                car_price = parts[-1]

                # Записываем данные в CSV
                writer.writerow([car_name, car_age, car_km, car_price])

            page += 1  # Переходим на следующую страницу

        except requests.exceptions.HTTPError as err:
            print(f"Произошла HTTP ошибка: {err}")
            break  # Прерываем цикл в случае HTTP ошибки
        except requests.exceptions.RequestException as e:
            print(f"Произошла ошибка запроса: {e}")
            break  # Прерываем цикл в случае других ошибок запроса

Слишком много запросов, ждем 10 секунд.
Слишком много запросов, ждем 10 секунд.
Слишком много запросов, ждем 10 секунд.
Слишком много запросов, ждем 10 секунд.
Слишком много запросов, ждем 10 секунд.
Слишком много запросов, ждем 10 секунд.
Слишком много запросов, ждем 10 секунд.
Слишком много запросов, ждем 10 секунд.
Слишком много запросов, ждем 10 секунд.
